# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 03: Model training & UI Exploration</span>

<span style="font-width:bold; font-size: 1.4rem;">In this last notebook, we will train a model on the dataset we created in the previous tutorial. We will train our model using standard Python and Scikit-learn, although it could just as well be trained with other machine learning frameworks such as PySpark, TensorFlow, and PyTorch. We will also show some of the exploration that can be done in Hopsworks, notably the search functions and the lineage. </span>

## **🗒️ This notebook is divided in 5 main sections:** 
1. **Loading the training data**
2. **Train the model**
3. **Register model to Hopsworks model registry**.
4. **Deploy the model on KServe behind Hopsworks for real-time inference requests**.
5. **Test model deployment and use model serving rest APIs**.

![tutorial-flow](../images/03_model.png)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

In [ ]:
feature_view = fs.get_feature_view("transactions_fraud_online_fv", 1)

## <span style="color:#ff5f27;"> ✨ Load Training Data </span>

First, we'll need to fetch the training dataset that we created in the previous notebook. We will use January - February data training and testing.

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression

train_jan_feb_x, train_jan_feb_y = feature_view.get_training_data(1)
test_mar_x, test_mar_y = feature_view.get_training_data(2)

## <span style="color:#ff5f27;"> 🏃 Train Model</span>

Next we'll train a model. Here, we set the class weight of the positive class to be twice as big as the negative class.

In [ ]:
import numpy as np
from sklearn.ensemble import IsolationForest

# fit the model
clf = IsolationForest()
clf.fit(train_jan_feb_x)

In [ ]:
# Train Predictions
y_pred_train = clf.predict(train_jan_feb_x)

In [ ]:
# Test Predictions
y_pred_test = clf.predict(test_mar_x)

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score
from matplotlib import pyplot
import seaborn as sn

%matplotlib inline

if_cm=confusion_matrix(test_mar_y, y_pred_test)
pd.DataFrame(if_cm)
df_cm = pd.DataFrame(if_cm, ['step', 'True Normal',  'True Fraud'],['Pred Normal', 'step', 'Pred Fraud'])
df_cm.drop(index="step",inplace=True)
df_cm.drop("step", axis=1, inplace=True)

pyplot.figure(figsize = (8,4))
sn.set(font_scale=1.4)#for label size
sn.heatmap(df_cm, annot=True,annot_kws={"size": 16},fmt='g')# font size

In [ ]:
if_cm=confusion_matrix(train_jan_feb_y, y_pred_train)
pd.DataFrame(if_cm)
df_cm = pd.DataFrame(if_cm, ['step', 'True Normal',  'True Fraud'],['Pred Normal', 'step', 'Pred Fraud'])
df_cm.drop(index="step",inplace=True)
df_cm.drop("step", axis=1, inplace=True)

pyplot.figure(figsize = (8,4))
sn.set(font_scale=1.4)#for label size
sn.heatmap(df_cm, annot=True,annot_kws={"size": 16},fmt='g')# font size

In [ ]:
from sklearn.metrics import f1_score
# Compute f1 score
metrics = {"fscore": f1_score(test_mar_y, y_pred_test, average='micro')}
metrics


## <span style="color:#ff5f27;"> Register model</span>

One of the features in Hopsworks is the model registry. This is where we can store different versions of models and compare their performance. Models from the registry can then be served as API endpoints.

Let's connect to the model registry using the HSML library from Hopsworks.

In [ ]:
mr = project.get_model_registry()

In [ ]:
import joblib
joblib.dump(clf, 'model.pkl')

The model needs to be set up with a Model Schema, which describes the inputs and outputs for a model.

A Model Schema can be automatically generated from training examples, as shown below.

In [ ]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(train_jan_feb_x)
output_schema = Schema(train_jan_feb_y)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

model_schema.to_dict()

In [ ]:
test_credit_card = [4467360740682089]
model = mr.sklearn.create_model(
    name="transactions_fraud_online_model",
    metrics=metrics,
    description="Isolation forest anomaly detection model",
    input_example = test_credit_card,
    model_schema=model_schema
)

model.save('model.pkl')

## <span style="color:#ff5f27;"> Deploy model</span>
### About Model Serving
Models can be served via KFServing or "default" serving, which means a Docker container exposing a Flask server. For KFServing models, or models written in Tensorflow, you do not need to write a prediction file (see the section below). However, for sklearn models using default serving, you do need to proceed to write a prediction file.

In order to use KFServing, you must have Kubernetes installed and enabled on your cluster.

### Create the Prediction File
In order to deploy a model, you need to write a Python file containing the logic to return a prediction from the model. Don't worry, this is usually a matter of just modifying some paths in a template script. An example can be seen in the code block below, where we have taken this Scikit-learn template script and changed two paths (see comments).

In [ ]:
%%writefile predict_example.py
import os
import numpy as np
import hsfs
import joblib

class Predict(object):

    def __init__(self):
        """ Initializes the serving state, reads a trained model"""        
        # get feature store handle
        fs_conn = hsfs.connection()
        self.fs = fs_conn.get_feature_store()
        
        # get feature views
        self.fv = self.fs.get_feature_view("transactions_fraud_online_fv", 1)
        
        # initialise serving
        self.fv.init_serving(1)

        # load the trained model
        self.model = joblib.load(os.environ["ARTIFACT_FILES_PATH"] + "/model.pkl")
        print("Initialization Complete")

    def predict(self, inputs):
        """ Serves a prediction request usign a trained model"""
        return self.model.predict(np.asarray(self.fv.get_feature_vector({"cc_num": inputs[0]})).reshape(1, -1)).tolist() # Numpy Arrays are not JSON serializable


If you wonder why we use the path Models/fraud_tutorial_model/1/model.pkl, it is useful to know that the Data Sets tab in the Hopsworks UI lets you browse among the different files in the project. Registered models will be found underneath the Models directory. Since we saved our model with the name fraud_tutorial_model, that's the directory we should look in. 1 is just the version of the model we want to deploy.

This script needs to be put into a known location in the Hopsworks file system. Let's call the file predict_example.py and put it in the Models directory.

In [ ]:
import os
dataset_api = project.get_dataset_api()

uploaded_file_path = dataset_api.upload("predict_example.py", "Models", overwrite=True)
predictor_script_path = os.path.join("/Projects", project.name, uploaded_file_path)

## Create the deployment
Here, we fetch the model we want from the model registry and define a configuration for the deployment. For the configuration, we need to specify the serving type (default or KFserving) and in this case, since we use default serving and an sklearn model, we need to give the location of the prediction script.

In [ ]:
# Use the model name from the previous notebook.
model = mr.get_model("transactions_fraud_online_model", version=1)

# Give it any name you want
deployment = model.deploy(
    name="fraudonlinemodeldeployment", 
    model_server="PYTHON",
    serving_tool="KSERVE",
    script_file=predictor_script_path
)

In [ ]:
print("Deployment: " + deployment.name)
deployment.describe()

#### The deployment has now been registered. However, to start it you need to run:

In [ ]:
deployment.start()

In [ ]:
deployment.get_logs()

## Using the deployment
Let's use the input example that we registered together with the model to query the deployment.

In [ ]:
data = {
    "inputs": model.input_example
}

deployment.predict(data)

In [ ]:
deployment.get_logs()

### Retrieving our deployment and making a prediction


In [ ]:
ms = project.get_model_serving()
fraud_detector = ms.get_deployment("fraudonlinemodeldeployment")

In [ ]:
prediction = fraud_detector.predict(data)

In [ ]:
prediction

## Stop Deployment
To stop the deployment we simply run:

In [ ]:
deployment.stop()

## 👾 StreamLit App


If you want to see interactive dashboards - use a **StreamLit App**.

Type the next commands in terminal to run a Streamlit App:

`cd {%path_to_hopsworks_tutorials%}/online_fraid`

`conda activate {%path_to_hopsworks_tutorials%}/miniconda/envs/hopsworks`

`python -m streamlit run streamlit_app.py`


## <span style="color:#ff5f27;"> 🎁  Wrapping things up </span>

In this module we introduced stream feature group, performed with training data that we have created from feature view and depoyed model in production.